In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [5]:
# Load in population data
county = pd.read_csv("data/county_pop.csv", header=4)
print(county.head())
county.columns = ["County", "Estimates Base", "Pop_2020", "Pop_2021", "Pop_2022"]
county = county.iloc[:-5, :]
county["County"] = county["County"].str[1:]
county[["County", "State"]] = county["County"].str.split(", ", expand=True)
county = county[
    ["State", "County", "Estimates Base", "Pop_2020", "Pop_2021", "Pop_2022"]
]
county.replace(",", "", regex=True, inplace=True)
county[["Estimates Base", "Pop_2020", "Pop_2021", "Pop_2022"]] = county[
    ["Estimates Base", "Pop_2020", "Pop_2021", "Pop_2022"]
].astype(int)
sc = county.groupby(["State", "County"])
jj = 0
county_pop = {}
for x, y in sc:
    county_pop[x[0] + ", " + x[1]] = y["Estimates Base"].values[0]
print(county.head())

              United States 331,449,520 331,511,512 332,031,554 333,287,557
0  .Autauga County, Alabama      58,802      58,902      59,210      59,759
1  .Baldwin County, Alabama     231,761     233,219     239,361     246,435
2  .Barbour County, Alabama      25,224      24,960      24,539      24,706
3     .Bibb County, Alabama      22,300      22,183      22,370      22,005
4   .Blount County, Alabama      59,130      59,102      59,085      59,512
     State          County  Estimates Base  Pop_2020  Pop_2021  Pop_2022
0  Alabama  Autauga County           58802     58902     59210     59759
1  Alabama  Baldwin County          231761    233219    239361    246435
2  Alabama  Barbour County           25224     24960     24539     24706
3  Alabama     Bibb County           22300     22183     22370     22005
4  Alabama   Blount County           59130     59102     59085     59512


In [8]:
registrations = pd.read_csv(
    "data/Motor_Vehicle_Registrations__by_vehicle_type_and_state_20231109.csv"
)
print(registrations.tail())
print("only up to 2018?")
registrations = registrations[["year", "state", "Auto"]]
recentReg = registrations[registrations["year"] > 2009]

# pip install openpyxl
vmts = pd.read_excel("data/10315_vmt_traveled_7-1-22.xlsx", header=2)
vmts.drop(vmts.tail(14).index, inplace=True)
vmts.drop(vmts.head(39).index, inplace=True)
vmts = vmts[["Year*", "Million Miles"]]
print("our vmts are")
print(vmts.tail())

state_pops = pd.read_excel("data/nst-est2019-01.xlsx", header=3, nrows=56)
state_pops = state_pops[5:56]

print(state_pops.tail())

vmt_perCap = pd.DataFrame()
vmt_perState = pd.DataFrame()

columns_to_use = state_pops.columns[3:13]
column_sum = state_pops[columns_to_use].sum(axis=1)
vmt_perCap[columns_to_use] = state_pops[columns_to_use].div(column_sum, axis=0)
vmt_perState[columns_to_use] = vmt_perCap[columns_to_use].multiply(
    np.array(vmts["Million Miles"])
)
vmt_perState.index = state_pops.iloc[:, 0]
print("vmt per state")
print(vmt_perState)

      year              state       Auto      Bus      Truck  Motorcycle
6067  2018           Virginia  3267735.0  34801.0  4106265.0    195845.0
6068  2018         Washington  2964939.0  23846.0  3928127.0    235501.0
6069  2018  West Virginia (2)   560118.0   3172.0  1069746.0     60683.0
6070  2018          Wisconsin  2087518.0  14892.0  3244241.0    336410.0
6071  2018            Wyoming   203546.0   4058.0   600452.0     28968.0
      year              state       Auto
864   2017            Montana        NaN
917   2017           Nebraska        NaN
1853  2017             Nevada        NaN
5613  2010            Alabama  2211550.0
5614  2010             Alaska   228407.0
...    ...                ...        ...
6067  2018           Virginia  3267735.0
6068  2018         Washington  2964939.0
6069  2018  West Virginia (2)   560118.0
6070  2018          Wisconsin  2087518.0
6071  2018            Wyoming   203546.0

[462 rows x 3 columns]
                                2010          

In [22]:
import re

# Extracting state name without extra information
recentReg["state"] = recentReg["state"].apply(
    lambda x: re.sub(r"\([^)]*\)", "", x).strip()
)
vmt_perState.index = vmt_perState.index.str.lstrip(".")

combined_data = pd.merge(
    recentReg, vmt_perState, how="inner", left_on=["state"], right_index=True
)

print(combined_data)

      year    state      Auto           2010           2011           2012  \
864   2017  Montana       NaN  283806.046034  288416.746603  288903.013968   
5639  2010  Montana  351574.0  283806.046034  288416.746603  288903.013968   
5690  2011  Montana  435575.0  283806.046034  288416.746603  288903.013968   
5741  2012  Montana  433264.0  283806.046034  288416.746603  288903.013968   
5792  2013  Montana  444707.0  283806.046034  288416.746603  288903.013968   
...    ...      ...       ...            ...            ...            ...   
5867  2014  Wyoming  216594.0  287727.546009  291908.275214  295128.619000   
5918  2015  Wyoming  206884.0  287727.546009  291908.275214  295128.619000   
5969  2016  Wyoming  211613.0  287727.546009  291908.275214  295128.619000   
6020  2017  Wyoming  193523.0  287727.546009  291908.275214  295128.619000   
6071  2018  Wyoming  203546.0  287727.546009  291908.275214  295128.619000   

               2013           2014           2015           201

<ipython-input-22-f3076a226e15>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recentReg['state'] = recentReg['state'].apply(lambda x: re.sub(r"\([^)]*\)", "", x).strip())


In [31]:
import pandas as pd
from sklearn.linear_model import LinearRegression

merged_data = pd.merge(recentReg, vmt_perState, left_on=["state"], right_index=True)
print(merged_data.head(200))
# Create a list of columns for regression
regression_cols = ["Auto_change"] + list(vmt_perState.columns)

# Perform linear regression for each year
for year in regression_cols[1:]:
    # Select data for the current year
    year_data = merged_data[["Auto_change", year]].dropna()

    # Extract features (X) and target variable (y)
    X = year_data[["Auto_change"]]
    y = year_data[year]

    # Create a linear regression model
    model = LinearRegression()

    # Fit the model
    model.fit(X, y)

    # Print the coefficients
    print(f"Year: {year}, Coefficient: {model.coef_[0]}, Intercept: {model.intercept_}")

                              year      state       Auto  Auto_change  \
864  1970-01-01 00:00:00.000002017    Montana        NaN          NaN   
5639 1970-01-01 00:00:00.000002010    Montana   351574.0          NaN   
5690 1970-01-01 00:00:00.000002011    Montana   435575.0      84001.0   
5741 1970-01-01 00:00:00.000002012    Montana   433264.0      -2311.0   
5792 1970-01-01 00:00:00.000002013    Montana   444707.0      11443.0   
...                            ...        ...        ...          ...   
5886 1970-01-01 00:00:00.000002015  Louisiana  1438568.0     -20772.0   
5937 1970-01-01 00:00:00.000002016  Louisiana  1425349.0     -13219.0   
5988 1970-01-01 00:00:00.000002017  Louisiana  1389436.0     -35913.0   
6039 1970-01-01 00:00:00.000002018  Louisiana  1389249.0       -187.0   
5632 1970-01-01 00:00:00.000002010      Maine   518779.0          NaN   

               2010           2011           2012           2013  \
864   283806.046034  288416.746603  288903.013968  2926

In [33]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assume 'recentReg' and 'vmt_perState' are your DataFrames

# Merge 'recentReg' with 'vmt_perState' using 'state'
merged_data = pd.merge(recentReg, vmt_perState, left_on=["state"], right_index=True)

# Create a list of columns for regression
regression_cols = ["Auto_change"] + list(vmt_perState.columns)

# Create an empty DataFrame to store regression results
regression_results = pd.DataFrame(columns=["state", "coefficient", "intercept"])

# Perform linear regression for each state
for state in merged_data["state"].unique():
    # Select data for the current state
    state_data = merged_data[merged_data["state"] == state].dropna()

    # Extract features (X) and target variable (y)
    X = state_data[["Auto_change"]]
    y = state_data[regression_cols[1:]].mean(axis=1)  # Take the mean across years

    # Create a linear regression model
    model = LinearRegression()

    # Fit the model
    model.fit(X, y)

    # Append results to the DataFrame
    regression_results = regression_results.append(
        {"state": state, "coefficient": model.coef_[0], "intercept": model.intercept_},
        ignore_index=True,
    )

# Display the regression results
print(regression_results)

                   state   coefficient      intercept
0                Montana -0.000000e+00  306436.761237
1               Nebraska -0.000000e+00  306359.960827
2                 Nevada -0.000000e+00  306648.324759
3                Alabama -0.000000e+00  306246.378472
4                 Alaska  0.000000e+00  306223.115615
5                Arizona -0.000000e+00  306611.066885
6               Arkansas -0.000000e+00  306272.694827
7             California  0.000000e+00  306362.245497
8               Colorado  0.000000e+00  306623.920769
9            Connecticut  0.000000e+00  306142.072170
10              Delaware -0.000000e+00  306435.287510
11  District of Columbia -5.340327e-32  306668.516557
12               Florida -0.000000e+00  306642.192841
13               Georgia  0.000000e+00  306475.678487
14                Hawaii -0.000000e+00  306271.641577
15                 Idaho -0.000000e+00  306630.601774
16              Illinois  0.000000e+00  306111.917468
17               Indiana  0.